In [60]:
# Initializing display and earthengine
from IPython.display import Image
%matplotlib inline

import ee
from ee import batch

ee.Initialize()

# importing ipygee for dynamic mapping
from ipygee import *

Map = Map() # from ipygee
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [61]:
# /// define a geometry called Geometry. This script is tested on a region in south korea
# /// script here https://code.earthengine.google.com/856588caf0ec3d680310d96386db71fb 
# /// I acknowledge and thanks the Google Earth Engine Developers mailing list for the help with this
# /// Non commerical code, purely for educational purposes
# /// A summer 2018 project by Andrew Cutts

In [62]:
geometry = ee.Geometry.Polygon(
        [[[126.1300964920049, 36.03156674990238],
          [126.1355896560674, 35.54587234645661],
          [127.0749207107549, 35.54587234645661],
          [127.0749207107549, 36.06709700011599]]]);
        
# // Landat 5 surface reflection data
L5coll = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').\
filter(ee.Filter.lt('CLOUD_COVER',5)).\
select(['B3', 'B2', 'B1']).\
filterBounds(geometry)

In [63]:
# // Landat 7 surface reflection data, fill in the gaps! See USGS pages for more info
def fill_im(image):
    filled1a = image.focal_mean(2, 'square', 'pixels', 1)
    return filled1a.blend(image)


L7coll = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').\
filter(ee.Filter.lt('CLOUD_COVER',5)).\
select(['B3', 'B2', 'B1']).\
filterBounds(geometry).\
map(fill_im)

In [64]:
# // Landat 8 surface reflection data, rename the band names. See USGS pages for more info
def rename_im(image):
    return image.rename(['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11']);


L8coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').\
filter(ee.Filter.lt('CLOUD_COVER',5)).\
filterBounds(geometry).\
map(rename_im).\
select(['B3', 'B2', 'B1']);

In [69]:
# // merge L5, L7 & L8
collection_merge = ee.ImageCollection(L5coll.merge(L7coll.merge(L8coll)));
# print (collection_merge)

# // create a list of years to be iterated over next..
years = ee.List.sequence(1984, 2018)
# //print (years)

# // create a collection with 1 image for each year
def merge_im(y):
    start = ee.Date.fromYMD(y, 1, 1)
    end = start.advance(12, 'month');
    return collection_merge.filterDate(start, end).reduce(ee.Reducer.median())


collectYear = ee.ImageCollection(years.map(merge_im))

# //print (collectYear)

# // count number of bands in each image, if 0 remove from image collection
def set_im(image):
      return image.set('count', image.bandNames().length())


nullimages = collectYear.map(set_im).filter(ee.Filter.eq('count', 3))
# //print(nullimages)

# // visualise the collection
def vis_im(image):
    return image.visualize(bands= ['B3_median', 'B2_median', 'B1_median'], min= 300, max= 10000)

finalCollection = nullimages.map(vis_im)

In [66]:
## Make 8 bit data
def convertBit(image):
    return image.uint8()  

## Convert bands to output video  
outputVideo = finalCollection.map(convertBit)

print("Starting to create a video")

## Export video to Google Drive
out = batch.Export.video.toDrive(outputVideo, 
                                 description='Netherlands_video_region_L8_time', 
                                 dimensions = 720, 
                                 framesPerSecond = 1, 
                                 region=geometry,
                                 maxFrames=10000)

## Process the image
process = batch.Task.start(out)

print("Process sent to cloud")



Starting to create a video
Process sent to cloud


In [70]:
# // add to map the first image in collection to check
median_1987 = finalCollection.first()
Map.addLayer(median_1987, {}, 'first image7');
# © 2020 GitHub, Inc.